<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/kobert/load_model_success(85).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
# 변경: 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9 MB 43 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 16.5 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595738 sha256=e5de3e0fdbcd1ed3076771bf6ca2ee7dc723f781c40957d689f2793bdd027012
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 22.7 MB/s 
     |████████████████████████████████| 754 kB 27.0 MB/s 
     |████████████████████████████████| 895 kB 64.1 MB/s 
     |████████████████████████████████| 3.0 MB 52.1 MB/s 


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-u169t1i3
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-u169t1i3
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12770 sha256=59a9d4f7f6fa196a4717a10a45c2b003a902f0d4dbd96a5063e2fc5a9382b087
  Stored in directory: /tmp/pip-ephem-wheel-cache-yc3pbv30/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768, # bert의 hidden layer 크기?
                #  num_classes=2,
                 num_classes=9, # 분류 class 크기
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes) #nn.Linear
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [5]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [7]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [8]:
## Setting parameters
max_len = 128 # max_length 512, but Considering Computing Resource => set 163
batch_size = 16 # GPU CUDA problem
warmup_ratio = 0.1
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate = 1e-5

In [9]:
import torch
device = torch.device('cpu')
model = torch.load('/content/drive/MyDrive/rsn_nlp_project/20210802model_95.pt', map_location=device)

In [132]:
import pandas as pd
import re

lucy_data = pd.read_csv('/content/drive/MyDrive/rsn_nlp_project/lucy_data0805_완.csv')

lucy_data['contents'] = lucy_data['contents'].str.replace("\(.*\)|\s-\s.*"," " ,regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("\[.*\]|\s-\s.*"," ",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("\<.*\>|\s-\s.*"," ",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("무단전재 및 재배포 금지"," ",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("무단 전재 및 재배포 금지"," ",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("©"," ",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("ⓒ"," ",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("저작권자"," ",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace(".* 기자", " ", regex=True) #기자 이름에서 오는 유사도 차단
lucy_data['contents'] = lucy_data['contents'].str.replace("사진 = .*", " ", regex=True) #사진 첨부 문구 삭제
lucy_data['contents'] = lucy_data['contents'].str.replace("사진=.*", " ", regex=True) #사진 첨부 문구 삭제
lucy_data['contents'] = lucy_data['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
lucy_data['contents'] = lucy_data['contents'].str.replace("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)", " ", regex=True) #이메일 주소에서 오는 유사도 차단
lucy_data['contents'] = lucy_data['contents'].str.replace("\n"," ")
lucy_data['contents'] = lucy_data['contents'].str.replace("\r"," ")
lucy_data['contents'] = lucy_data['contents'].str.replace("\t"," ")
lucy_data['contents'] = lucy_data['contents'].str.replace( "\’" , "", regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace('\"', "",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("[ ]{2,}"," ",regex=True)

lucy_data[:3]

,id,contents,category
0,DN000000008778200425,공정소득은 코로나 이후 악화되는 자형 양극화에 대비하자는 것 반박 야권의 대선 주...,정치
1,DN000000008808263362,공동대표 및 정책자문위원 등 임명장 수여 내 삶을 지켜주는 나라 신복지 충남포럼 ...,정치
2,DN000000008791388703,바르셀로나 참가 인텔 기가테라 커뮤니케이션즈 를 오픈랜 생태계를 통하여 구축 및 ...,IT/과학


In [133]:
lucy_data['category_val'] = lucy_data['category']

In [134]:
label_dict = {'0': 'IT/과학',
 '1': '경제',
 '2': '문화',
 '3': '미용/건강',
 '4': '사회',
 '5': '생활',
 '6': '스포츠',
 '7': '연예',
 '8': '정치'}

for key, value in label_dict.items():
  print(value)
  lucy_data['category'] = lucy_data['category'].str.replace(value, key)

lucy_data.to_csv('/content/drive/MyDrive/rsn_nlp_project/lucy_data0805.txt', sep = '\t' , index = False)

IT/과학
경제
문화
미용/건강
사회
생활
스포츠
연예
정치


In [135]:
lucy_data[:5]

,id,contents,category,category_val
0,DN000000008778200425,공정소득은 코로나 이후 악화되는 자형 양극화에 대비하자는 것 반박 야권의 대선 주...,8,정치
1,DN000000008808263362,공동대표 및 정책자문위원 등 임명장 수여 내 삶을 지켜주는 나라 신복지 충남포럼 ...,8,정치
2,DN000000008791388703,바르셀로나 참가 인텔 기가테라 커뮤니케이션즈 를 오픈랜 생태계를 통하여 구축 및 ...,0,IT/과학
3,DN000000008853835944,더불어민주당 신영대 의원 이 민주당의 혁신과 흥행을 담당할 대선경선기획단에 인선됐...,8,정치
4,DN000000008834209192,지난 일 미 멕시코 국경지대인 텍사스주 엘패소와 멕시코 시우대드후아레스 사이에서 ...,4,사회


{0: 'IT/과학',
 1: '경제',
 2: '문화',
 3: '미용/건강',
 4: '사회',
 5: '생활',
 6: '스포츠',
 7: '연예',
 8: '정치'}

In [136]:
test_set = nlp.data.TSVDataset("/content/drive/MyDrive/rsn_nlp_project/lucy_data0805.txt", field_indices=[1,2], num_discard_samples=1)

In [137]:
test_set[0]

[' 공정소득은 코로나 이후 악화되는 자형 양극화에 대비하자는 것 반박 야권의 대선 주자인 유승민 바른정당 전 대표가 자신의 공정소득 이론을 비판한 이재명 경기도지사를 향해 앞으로 토론을 하려면 뭐든지 똑바로 알고 똑바로 인용하라 노벨경제학상 수상자들의 말까지 왜곡해가며 나를 비난하려 애쓰는 모습이 안타깝다 고 비판했다 를 비판하면서 수백 수천만원을 그것도 일을 적게 할수록 더 많이 주자는 것 이라고 했다 고 지적했다 유 전 대표는 자신은 이런 말을 한 적이 없다며 나는 공정소득의 원리는 단순하고 분명하다 고소득층은 세금을 내고 저소득층은 보조금을 받는 것이다 이는 사회복지의 원칙이고 상식이다 공정소득은 코로나 이후 악화되는 자형 양극화에 대비하자는 것이다 고 했다 고 설명했다 아울러 이 지사가 노벨경제학상을 수상한 바네르지 뒤플로 교수 부부가 기본소득을 찬성했다 고 주장한 것에 대해서도 잘못된 인용이자 왜곡이다 이 지사의 잘못된 인용이 고의로 거짓말을 지어낸 것인지 아니면 책도 안 읽어본 참모들이 잘못 써준대로 한 것인지 알 수 없다 고 일갈했다 이어 그는 노벨경제학상을 받은 사람들 중에는 기본소득에 찬성하는 사람도 있고 공정소득 에 찬성하는 사람도 있다 기본소득에 찬성하는 사람들도 그들이 말하는 기본소득이 전국민 대상이 아니라 저소득층이나 일부 국민들 대상인 경우가 많다 며 상대방이 하지도 않은 말을 자기 맘대로 지어내어 덮어씌우는 것은 거짓말쟁이들이나 하는 행태다 고 덧붙였다 ',
 '8']

In [138]:
lucy_data['predict'] = -1
lucy_data['predict_tag'] = 'a'

In [139]:
test_set = BERTDataset(test_set, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=9)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  out_val = torch.argmax(out).cpu().numpy()
  out_tag = ''
  print(lucy_data['contents'][batch_id][1:50],'...')
  if(out_val == 0):
    print('IT/과학')
    out_tag = 'IT/과학'
  elif(out_val == 1):
    print('경제')
    out_tag = '경제'
  elif(out_val == 2):
    print('문화')
    out_tag = '문화'
  elif(out_val == 3):
    print('미용/건강')
    out_tag = '미용/건강'
  elif(out_val == 4):
    print('사회')
    out_tag = '사회'
  elif(out_val == 5):
    print('생활')
    out_tag = '생활'
  elif(out_val == 6):
    print('스포츠')
    out_tag = '스포츠'
  elif(out_val == 7):
    print('연예')
    out_tag = '연예'
  elif(out_val == 8):
    print('정치')
    out_tag = '정치'
  lucy_data['predict'][batch_id] = out_val
  lucy_data['predict_tag'][batch_id] = out_tag

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 9 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


공정소득은 코로나 이후 악화되는 자형 양극화에 대비하자는 것 반박 야권의 대선 주자인 유 ...
정치


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


공동대표 및 정책자문위원 등 임명장 수여 내 삶을 지켜주는 나라 신복지 충남포럼 오전 시 ...
정치
바르셀로나 참가 인텔 기가테라 커뮤니케이션즈 를 오픈랜 생태계를 통하여 구축 및 혁신 는 ...
IT/과학
더불어민주당 신영대 의원 이 민주당의 혁신과 흥행을 담당할 대선경선기획단에 인선됐다 더불 ...
정치
지난 일 미 멕시코 국경지대인 텍사스주 엘패소와 멕시코 시우대드후아레스 사이에서 떨어져  ...
사회
불법 총기류 고스트건 제작한 일당 검거 일 부산경찰청 강력범죄수사대에 따르면 불법으로 총 ...
사회
과학적 연구로 증거 찾는 수밖에 에 조사 맡기면 안돼 영국 해외정보국 전 수장은 중국이  ...
사회
기자 입건하거나 수사한 사실 없어 김진욱 고위공직자범죄수사처장이 일 정부과천청사 공수처로 ...
사회
용납될 수 없는 행위 시인 독립 조사 계획 제공 연합뉴스 올림픽 불참을 선언한 호주 수영 ...
사회
주기적 발병 미국서 항상 어느 정도의 확산 있을 것 팬데믹 초기와는 다를 것 백신 접종률 ...
미용/건강
충혼서린 현충원 둘레길에서 시민 위한 올곧은 정책 필 것 허태정 시장 호국보훈의 달 맞아 ...
사회
코로나 팬데믹으로 자가 건강관리 중요해져 처방전 없어도 사후피임약 등 구입할 수 있어야  ...
미용/건강
가평군이 향토인재 육성을 위해 운영하고 있는 서울소재 가평장학관 입사생들을 대상으로 자라 ...
사회
전해철 행정안전부 장관이 일 오후 광주 동구 학동 명의 사상자가 발생한 붕괴 사고 현장에 ...
사회
한국전 참전용사들을 기리며 한국전 추모패가 있는 런던 세인트폴 대성당에서 헌화하고 예배를 ...
생활
부터 전국 거의 모든 지역에서 실외 마스크 착용 의무를 해제했다 신종 코로나바이러스 감염 ...
사회
정유지에 대한 관심이 커지고 있다 일 밤 시 방송된 조선 사랑의 콜센타 에서는 특집으로  ...
연예
한화시스템 도서지역 장병에 언택트 진로 멘토링 이 육군 사단 진도대대 장병 여 명을 대상 ...
사회
봄배추 수매 현장 애로사항 청취 박석배 

In [140]:
#얼마나 타겟의 값을 잘 맞추었는지 평가하는 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [141]:
model.eval() # 평가 모드로 변경
      
test_acc = 0.0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)
    print(test_acc)
print("test acc {}".format(test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 9 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
14.0
14.0
15.0
16.0
17.0
18.0
19.0
19.0
20.0
21.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
34.0
35.0
36.0
37.0
38.0
39.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
51.0
52.0
53.0
54.0
54.0
55.0
56.0
57.0
58.0
59.0
59.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
71.0
72.0
73.0
74.0
75.0
75.0
75.0
76.0
76.0
76.0
77.0
77.0
78.0
78.0
78.0
79.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
93.0
94.0
95.0
96.0
97.0
97.0
98.0
99.0
100.0
101.0
101.0
102.0
103.0
104.0
104.0
104.0
105.0
106.0
107.0
108.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
127.0
128.0
129.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
142.0
142.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157

In [142]:
lucy_data.to_csv('/content/drive/MyDrive/rsn_nlp_project/lucy_data0805.csv', encoding='euc-kr')